In [22]:
from bs4 import BeautifulSoup
import unicodedata
import requests
import re

In [23]:
def get_characters(gra):
    url = f'https://gothic.fandom.com/pl/wiki/Kategoria:{gra}'
    response = requests.get(url)
    html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')

    links = soup.find_all('a', class_='category-page__member-link')

    links_array = []

    for link in links:
        links_array.append(link['href'])

    clean_links_array = []

    for link in links_array:
        clean_link = link.replace('/pl/wiki/', '')
        clean_links_array.append(clean_link)
    return clean_links_array

In [24]:
def usun_nawiasy(input_list):
    return [re.sub(r'\(.*?\)', '', item) for item in input_list]

def insert_l(input_list): 
    return [item.replace("opata","lopata").replace("Dlugi luk)","Dlugi luk").replace("luk polowy)","luk polowy").replace("/",",").replace("luk polowy)","luk polowy").replace("achy skazanca","lachy").replace("luk polowy)","luk polowy").replace("Obywatelka","Obywatel").replace("Kujacy","Klujacy").replace(" (od II rozdziau","").replace("potorareczny","poltorareczny").replace("Chopski","Chlopski").replace("Obywatele Khorinis","Obywatel").replace("Bero","Berlo").replace("Zado","Zadlo").replace("Czonek Wodnego Kregu","Wodny Krag").replace("ponocna","polnocna").replace("amacz","Lamacz").replace("omotacz","Lomotacz").replace("Dugi","Dlugi").replace("owca","Lowca").replace("owcy","Lowca").replace("czonek","czlonek").replace("Paka","Palka").replace(" ky"," kly").replace("Zodziej)","Zlodziej") for item in input_list]

def insert_luk(input_list):
    return [item.replace("uk","luk")for item in input_list]

def del_zaleznosc_growa(input_list):
    return [item.replace("Gothic II: Noc Kruka: ", "").replace("Gothic II: ", "").replace("Gothic: ", "") for item in input_list]
    
def clean_list(input_list):
    not_included=["Nieznana", "Brak", "ArcaniA: Upadek Setarrif", "Gothic: Der Comic", "Gothic 3: Zmierzch Bogow (wzmianki)","Zalezne od wyboru gracza"]
    cleaned_list = list(set(input_list))
    cleaned_list = [unicodedata.normalize('NFKD', item.text.strip()).encode('ascii', 'ignore').decode('utf-8') for item in cleaned_list]
    return [item for item in cleaned_list if item not in not_included]


def sort_gothic_games(games):
    gothic_order = ["Gothic", "Gothic II","Gothic II: Noc Kruka", "Gothic 3", "Gothic 3: Zmierzch Bogow", "ArcaniA"]
    sorted_games = sorted([game for game in games if game in gothic_order], key=lambda x: gothic_order.index(x))
    return sorted_games

def clear_name(input_list):
    return [item.replace(" (Gothic)", "").replace(" (Gothic II: Noc Kruka)", "") for item in input_list]


In [25]:
def procces_weapon(input_list):
    input_list=usun_nawiasy(input_list)
    input_list=del_zaleznosc_growa(input_list)
    input_list=insert_luk(input_list)
    input_list=insert_l(input_list)
    return input_list

def procces_armour(input_list):
    input_list = usun_nawiasy(input_list)
    input_list=insert_l(input_list)
    input_list=usun_nawiasy(input_list)
    return input_list

def procces_przynaleznosc(input_list):
    input_list=clean_list(input_list)
    input_list=insert_l(input_list)
    return input_list

def procces_name(input_list):
    input_list = usun_nawiasy(input_list)
    input_list=input_list[0].replace("Zy", "Zly")
    input_list=''.join(c for c in unicodedata.normalize('NFKD', input_list.strip()).encode('ascii', 'ignore').decode('utf-8')).replace("Zy", "Zly")
    return input_list

def procces_game(input_list):
    input_list = sort_gothic_games(input_list)
    return input_list

In [26]:
def make_req(url):
    url = f'https://gothic.fandom.com/pl/wiki/{url}'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup
    
def scrap(soup):
    posiadana_zbroja = clean_list(soup.find_all(attrs={"data-line": 'Posiadana zbroja'}))
    if not posiadana_zbroja:
        posiadana_zbroja=["brak"]

    posiadana_bron = clean_list(soup.find_all(attrs={"data-line": "Posiadana broń"}))
    if not posiadana_bron:
        posiadana_bron=["brak"]

    wystepowanie = clean_list(soup.find_all(attrs={"data-line": 'Występowanie'}))
    if not wystepowanie:
        wystepowanie=["brak"]

    imie = soup.find(class_='pi-title-icon').text
    
    przynaleznosc = []
    przeynaleznosc_divs = soup.find_all('div', attrs={'data-line': 'Gildia'})

    for div in przeynaleznosc_divs:
        link = div.find('a')
        if link:
            przynaleznosc.append(link.text.strip())
    przynaleznosc = [BeautifulSoup(tag, 'html.parser') for tag in przynaleznosc]

    if not przynaleznosc:
        przynaleznosc=["brak"]

    return[posiadana_zbroja,posiadana_bron,wystepowanie,imie,przynaleznosc]

def print_query(imie,przynaleznosc,wystepowanie_list,bron_list,zbroja_list):
    print(f"INSERT INTO character(imie, przynaleznosc, wystepowanie, bron, zbroja) VALUES ('{imie}', '{{{', '.join(przynaleznosc)}}}', '{{{', '.join(wystepowanie_list)}}}', '{{{', '.join(bron_list)}}}', '{{{', '.join(zbroja_list)}}}');")
    return 0

In [27]:
G1    =get_characters("Postacie_z_Gothic")
G2_1  =get_characters("Postacie_z_Gothic_II")
G2_2  =get_characters("Postacie_z_Gothic_II?from=Potężny+Krasnolud+z+Kosmosu")
G2_nk =get_characters("Postacie_z_Gothic_II:_Noc_Kruka")
G3_1  =get_characters("Postacie_z_Gothic_3")
G3_2  =get_characters("Postacie_z_Gothic_3?from=Konrad")
G3_3  =get_characters("Postacie_z_Gothic_3?from=Wyjątkowo+ochoczy+niewolnik")

lista_postaci =G1 + G2_1 + G2_2 + G2_nk + G3_1 + G3_2 + G3_3 

# G1=["Fortuno","Ian","Scatty","Mordrag","Blizna","Cor_Kalom","Stone","Bloodwyn","Buster","Bullit","Cipher","Corristo","Cavalorn","Cronos","Cor_Kalom","Cord","Dexter","Diego","Drax","Dusty","Fisk","Fletcher","Gomez","Gorn","Graham","Huno","Kruk","Kyle","Lares","Lee","Lester","Lewus","Merdarion","Myxir","Milten","Nefarius","Nek","Riordian","Ryżowy_Książę","Rączka","Santino","Saturas","Snaf","Szakal_(Gothic)","Thorus","Ur-Shak","Wilk","Wrzód","Xardas","Zły","Świstak"]
# G2=["Ignaz", "Gritta", "Gerbrandt", "Moe", "Serpentes", "Sentenza","Abuyin","Akil","Attila","Baltram","Bartok","Onar","Bengar","Bennet","Boltan","Bosper","Bullko","Canthar","Constantino","Cornelius","Daron","Lord_Andre","Cor_Angar","Coragon","Dexter","Fernando","Gerold","Harad","Igaraz","Isgaroth","Ingmar","Jarvis","Jesper","Jora","Lehmar","Lobart","Lutero","Marcos","Matteo","Mika","Lord_Hagen","Lothar","Nadja","Nagur","Orlan","Peck","Parlan","Pablo","Pyrokar","Ramirez","Rangar","Regis","Rengaru","Rupert","Sara","Sagitta","Sekob","Sonja","Sylvio","Talamon","Thorben","Torlof","Valentino","Vanja","Wulfgar","Zuris"]
# G2_nk=["Eremita","Greg","Henry","Jack Aligator","Esteban","Quarhodron","Izabela","Vatras","Skip_(Gothic_II:_Noc_Kruka)"]
# lista_postaci =G1+G2+G2_nk


lista_postaci=list(set(lista_postaci))

postacie_do_usuniecia = ["Rhobar_I_Święty","Hodges","Yasmin","Uczen czarnego maga","Woczega","Uczeń_czarnego_maga","Hodges_(zmarły_obywatel)","Mr.Item","Uwe","Andre","Viktorus_Stahl","Oskar_Sorn""Veranim_Sadea","Serano_Ukara","Bjorn","Snoelk","Kairo","Uthar_Lichtbringer""Baal_Tyon","Seb","Sascha","Hodges","Zahra","Teodor","_(Gothic_II)","Simbus","Dietmar_Ukara","Lazar","Wilfied_Ukara","Uthar_Seranis""Uwe","Wujo_Dobra_Rada","Rhobar_I_Swiety","Sagitta","Talbin","Uczen_czarnego_maga","Włóczęga","Niclas","Kati","Postacie_w_Gothic_II","Dragomir","Engrom","Ork_pu%C5%82kownik_z_wyspy_Khorinis","Cassia","Goniec","M%C4%85%C5%BC_Gritty","Oelk_(zmar%C5%82y_obywatel)","Ernesto_Ortoj","Gaan","Hosh-Pak","NicoDE","Chase","Iotar","Izolda","Najwy%C5%BCsza_Rada","Jergan","Jorgen","Archol","Grom_(my%C5%9Bliwy)","Inubis","Ork_pułkownik_z_wyspy_Khorinis","Ork_pułkownik_z_wyspy_Irdorath","Oelk_(zmarły_obywatel)","Najwyższa_Rada","Mąż_Gritty","Mihai","Midos","Michael","Marta_Ukara","Mart_Mulgo","Marlas","Klucznik","Ivan_(Gothic_II)","Inspektor_poziomu","Hodges_(zmarły_obywatel)","Herszt_bandy","Heristun","Henrich_von_Stahl","Fehta","Ernesto_Ortoj""Dominique","Dex_Cantionis","Czarny_mag_z_Dworu_Irdorath","Brat_Orika","Bonka","Bertran","Bandyci_z_latarni_morskiej","Bandyta_(strażnik_jaskini_Brago)","Asub_Ukara","Arthag_Amashrog","Anieb","Andre_(zmarły_obywatel)","Hosh","Onurb","Bonka","Michael","Ork_pu%C5%82kownik_z_wyspy_Irdorath","Lucia","Izabela","Trokar","Aya","Pirat_(kucharz)","Bandyci_Kruka","Wielki_Uczony","Eremita","W%C3%B3dz_hordy","Kucharz_w_kopalni_z%C5%82ota","Stra%C5%BCnik_w_kopalni_z%C5%82ota","Postacie_w_Gothic_II:_Noc_Kruka","Stra%C5%BCnik_niewolnik%C3%B3w","Ochroniarze_Estebana","Khardimon","Bezimienny","Apokaliptyczni_stra%C5%BCnicy_%C5%9Bwi%C4%85tynni","W%C5%82%C3%B3cz%C4%99dzy_(Gothic)","W%C4%85%C5%BC_(Gothic)","Fanatyczni_stra%C5%BCnicy_%C5%9Bwi%C4%85tynni","Stra%C5%BCnik_przej%C5%9Bcia_na_tereny_ork%C3%B3w","Op%C4%99tani_nowicjusze","Mieszacze","Wykidaj%C5%82owie","Fanatyczni_stra%C5%BCnicy_%C5%9Bwi%C4%85tynni","Stra%C5%BCnik_przej%C5%9Bcia_na_tereny_ork%C3%B3w","%C5%BB%C3%B3%C5%82ty_Pfeiffer","Korbacz","%C5%81owca_trolli","Jednoro%C5%BCec","Y%27Berion","Kowal_Tomasz","Steve_(In_Extremo)","Łowca_trolli","Waz","Omid","Tarrok","Serafia","Dr_Pymonte","Żółty_Pfeiffer","Nadzorca","Jednorożec","Guy","Lutter","Thomas_the_Forger","Balam","Sira","Natalia","Steve","Chani","The_Flail","Flex","Charlotte","In_Extremo","Velaya","Rhobar_II","Bandziory","Stra%C5%BCnicy_mostu","Stra%C5%BCnik_%C5%9Bwi%C4%85tynny_(Gothic)",'Barthos','Bergmar','Gellon','Genera%C5%82_(w%C5%82a%C5%9Bciciel_Uriziela)','Korgur_Trzy_Palce','Lord_Tymorisin','Lukkor','Nieznajomy_(posta%C4%87)','Postacie_w_Gothic','%C5%BBona_Rhobara_II']

for postac in postacie_do_usuniecia:
    if postac in lista_postaci:
        lista_postaci.remove(postac)

In [28]:
for postac in lista_postaci:
    try:
        soup=make_req(postac)
        [posiadana_zbroja,posiadana_bron,wystepowanie,imie,przynaleznosc]=scrap(soup)

        imie=procces_name([imie])
        wystepowanie=procces_game(wystepowanie)
        przynaleznosc=procces_przynaleznosc(przynaleznosc)
        posiadana_bron=procces_weapon(posiadana_bron)
        posiadana_zbroja=procces_armour(posiadana_zbroja)

        zbroja_list = [item for item in posiadana_zbroja]
        bron_list = [item for item in posiadana_bron]
        wystepowanie_list = [item for item in wystepowanie]

        print_query(imie,przynaleznosc,wystepowanie_list,bron_list,zbroja_list)
    except:
        pass

INSERT INTO character(imie, przynaleznosc, wystepowanie, bron, zbroja) VALUES ('Daro', '{Asasyn}', '{Gothic 3}', '{2x miecz asasyna}', '{Pancerz asasyna}');
INSERT INTO character(imie, przynaleznosc, wystepowanie, bron, zbroja) VALUES ('Lee', '{Zabojca orkow, Najemnik, Paladyn}', '{Gothic, Gothic II, Gothic 3, Gothic 3: Zmierzch Bogow}', '{Topor Lee, Miecz dwureczny, Kiepski miecz dwureczny, Dlugi luk}', '{Pancerz paladyna, Ciezka zbroja najemnika, Ciezki pancerz Nordmarczyka, Myrtanski kirys, Ciezki pancerz najemnika}');
INSERT INTO character(imie, przynaleznosc, wystepowanie, bron, zbroja) VALUES ('Rega', '{Farmerka}', '{Gothic II}', '{Laga}', '{Suknia farmerki}');
INSERT INTO character(imie, przynaleznosc, wystepowanie, bron, zbroja) VALUES ('Serpentes', '{Mag ognia}', '{Gothic II}', '{brak}', '{Ciezka szata ognia}');
INSERT INTO character(imie, przynaleznosc, wystepowanie, bron, zbroja) VALUES ('Szef kopalni', '{Herszt}', '{Gothic 3}', '{Krush Varok}', '{brak}');
INSERT INTO charac